In [10]:
import numpy as np
import pandas as pd
from aocd import get_data, submit
from utils import magnitude

DAY = 18
YEAR = 2021

data = get_data(day=DAY, year=YEAR)
# print(data)

In [11]:
def process(data):
    return [eval(d) for d in data.split("\n")]


data = process(data)

# Part 1

In [12]:
class Node:
    def __init__(self, data, depth=0, parent=None):
        self.left = None
        self.right = None
        self.depth = depth
        self.parent = parent
        self.value = None

        if isinstance(data, int):
            self.value = data
        else:
            self.left = Node(data[0], depth=depth + 1, parent=self)
            self.right = Node(data[1], depth=depth + 1, parent=self)

    def __repr__(self):
        if self.value is not None:
            return str(self.value)
        return f"{[self.left, self.right]}"

    def add(self, data):
        return Node([self.data(), data])

    def data(self):
        return calc_data(self)

    def reduce(self):
        reduce_node(self)
        return Node(self.data())

    def explode(self):
        explode_node(self)
        return Node(self.data())

    def split(self):
        split_node(self)
        return Node(self.data())

    def magnitude(self):
        return magnitude(self)


def find_nearest_leaf_node(node, left=True):
    parent = node.parent
    if parent is None:
        return None

    # going up
    prev = node
    while True:
        if parent is None:
            return None
        if (left and parent.left == prev) or (not left and parent.right == prev):
            prev = parent
            parent = parent.parent
        else:
            break

    # going down:
    leaf = parent.left if left else parent.right
    while (left and leaf.right is not None) or (not left and leaf.left is not None):
        leaf = leaf.right if left else leaf.left

    return leaf


def explode_node(node, exploded=False):
    if not node or exploded:
        return exploded

    if node.depth == 4 and node.value is None and not exploded:
        left = find_nearest_leaf_node(node, True)
        right = find_nearest_leaf_node(node, False)
        if left is not None:
            left.value += node.left.value
        if right is not None:
            right.value += node.right.value
        node.left, node.right, node.value = None, None, 0
        return True

    exploded = explode_node(node.left, exploded)
    exploded = explode_node(node.right, exploded)
    return exploded


def split_node(node, splitted=False):
    if not node or splitted:
        return splitted

    if node.value is not None and node.value >= 10 and not splitted:
        node.left = Node(node.value // 2, depth=node.depth + 1, parent=node)
        node.right = Node((node.value + 1) // 2, depth=node.depth + 1, parent=node)
        node.value = None
        return True

    splitted = split_node(node.left, splitted)
    splitted = split_node(node.right, splitted)
    return splitted


def reduce_node(node):
    while True:
        exploded = explode_node(node)
        if not exploded:
            splitted = split_node(node)
            if not splitted:
                break


def calc_data(node):
    if node.value is not None:
        return node.value

    return [calc_data(node.left), calc_data(node.right)]


def magnitude(node):
    if node.value is not None:
        return node.value
    else:
        left = magnitude(node.left)
        right = magnitude(node.right)
        return 3 * left + 2 * right

In [13]:
from utils import explode as explode_str
from utils import reduce_number
from utils import split as split_str

In [14]:
total = Node(data[0])
for d in data[1:]:
    total = total.add(d).reduce()

result = total.magnitude()
result

4323

In [4]:
# submit(result, part="a", day=DAY, year=YEAR)

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


<Response [200]>

# Part 2

In [15]:
values = []
for idx in range(len(data)):
    for jdx in range(idx + 1, len(data)):
        values.append(Node(data[idx]).add(data[jdx]).reduce().magnitude())
        values.append(Node(data[jdx]).add(data[idx]).reduce().magnitude())

result = values[np.argmax(values)]
result

4749

In [17]:
# submit(result, part="b", day=DAY, year=YEAR)

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 18! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


<Response [200]>